# 02 — Market Capitalisation and Inputs

This notebook:
- Loads shares outstanding + debt (from fundamentals.csv)
- Loads price data for each ticker
- Extracts Adjusted Close at FY-end (≤ fy_end from config)
- Computes market value of equity E = price × shares
- Saves `marketcap_FY2025.csv`
- Prepares combined input for Merton PD calculation

In [25]:
import sys
import os

# Add project root to Python path
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("."))  

# If src is directly under your notebook folder:
sys.path.append(os.path.abspath("./src"))

In [26]:
import pandas as pd
from pathlib import Path

from src.utils import load_config, ensure_folder, read_fundamentals
from src.market_cap import compute_marketcap_from_fundamentals

cfg = load_config("../config.yaml")
ensure_folder(cfg["output_folder"])

tickers = cfg["tickers"]
price_folder = cfg["price_data_folder"]
fy_end = pd.to_datetime(cfg["fy_end"])

In [27]:
fund = read_fundamentals(cfg["fundamentals_file"])

fund  

{'SBIBANK': {'shares_outstanding': 8924620034,
  'short_term_debt': 26257164700000,
  'long_term_debt': 39885442200000,
  'source': 'AR2025',
  'notes ': nan,
  'Unnamed: 6': nan,
  'Unnamed: 7': nan},
 'BANKBARODA': {'shares_outstanding': 5171362179,
  'short_term_debt': 11301960400000,
  'long_term_debt': 14476385300000,
  'source': 'AR2025',
  'notes ': nan,
  'Unnamed: 6': nan,
  'Unnamed: 7': nan},
 'CANBK': {'shares_outstanding': 9076562500,
  'short_term_debt': 10072609700000,
  'long_term_debt': 25722651200000,
  'source': 'AR2025',
  'notes ': nan,
  'Unnamed: 6': nan,
  'Unnamed: 7': nan},
 'HDFCBANK': {'shares_outstanding': 5105325797,
  'short_term_debt': 402332200000,
  'long_term_debt': 32224695700000,
  'source': 'AR2025',
  'notes ': nan,
  'Unnamed: 6': nan,
  'Unnamed: 7': nan},
 'ICICIBANK': {'shares_outstanding': 3564037855,
  'short_term_debt': 6187340900000,
  'long_term_debt': 11151521900000,
  'source': 'AR2025',
  'notes ': nan,
  'Unnamed: 6': nan,
  'Unnamed:

In [28]:
# --- Compute Face Value of Debt (D) ---
fund_df = pd.DataFrame.from_dict(fund, orient="index").reset_index()
fund_df = fund_df.rename(columns={"index": "ticker"})

fund_df

,ticker,shares_outstanding,short_term_debt,long_term_debt,source,notes,Unnamed: 6,Unnamed: 7
0,SBIBANK,8924620034,26257164700000,39885442200000,AR2025,NaN,NaN,NaN
1,BANKBARODA,5171362179,11301960400000,14476385300000,AR2025,NaN,NaN,NaN
2,CANBK,9076562500,10072609700000,25722651200000,AR2025,NaN,NaN,NaN
3,HDFCBANK,5105325797,402332200000,32224695700000,AR2025,NaN,NaN,NaN
4,ICICIBANK,3564037855,6187340900000,11151521900000,AR2025,NaN,NaN,NaN
5,AXISBANK,3098620347,3581757300000,11410175700000,AR2025,NaN,NaN,NaN
6,KOTAKBANK,1988519342,6129009600000,9336198400000,AR2025,NaN,NaN,NaN
7,INDUSINDBK,779445161,2848660500000,3045799500000,AR2025,NaN,NaN,NaN
8,BAJFINANCE,6208203435,1085765100000,1683317300000,AR2025,NaN,NaN,NaN
9,PNB,11521086957,5895063500000,10608938500000,AR2025,NaN,NaN,NaN


In [ ]:
for col in ["short_term_debt", "long_term_debt"]:
    if col not in fund_df.columns:
        print(f"⚠️ Missing column: {col}, creating with zeros")
        fund_df[col] = 0

fund_df["short_term_debt"] = pd.to_numeric(fund_df["short_term_debt"], errors="coerce").fillna(0)
fund_df["long_term_debt"]  = pd.to_numeric(fund_df["long_term_debt"], errors="coerce").fillna(0)

# Merton/KMV approximation: D = ST + LT
fund_df["debt_facevalue"] = fund_df["short_term_debt"] + fund_df["long_term_debt"]

fund_df[["ticker", "short_term_debt", "long_term_debt", "debt_facevalue"]]


,ticker,short_term_debt,long_term_debt,debt_facevalue
0,SBIBANK,26257164700000,39885442200000,4.619989e+13
1,BANKBARODA,11301960400000,14476385300000,1.854015e+13
2,CANBK,10072609700000,25722651200000,2.293394e+13
3,HDFCBANK,402332200000,32224695700000,1.651468e+13
4,ICICIBANK,6187340900000,11151521900000,1.176310e+13
5,AXISBANK,3581757300000,11410175700000,9.286845e+12
6,KOTAKBANK,6129009600000,9336198400000,1.079711e+13
7,INDUSINDBK,2848660500000,3045799500000,4.371560e+12
8,BAJFINANCE,1085765100000,1683317300000,1.927424e+12
9,PNB,5895063500000,10608938500000,1.119953e+13


In [30]:
processed_fund_path = Path(cfg["output_folder"]) / "fundamentals_processed.csv"
fund_df.to_csv(processed_fund_path, index=False)

print("✓ Processed fundamentals (with D) saved to:", processed_fund_path)
fund_df

✓ Processed fundamentals (with D) saved to: ../outputs/fundamentals_processed.csv


,ticker,shares_outstanding,short_term_debt,long_term_debt,source,notes,Unnamed: 6,Unnamed: 7,debt_facevalue
0,SBIBANK,8924620034,26257164700000,39885442200000,AR2025,NaN,NaN,NaN,4.619989e+13
1,BANKBARODA,5171362179,11301960400000,14476385300000,AR2025,NaN,NaN,NaN,1.854015e+13
2,CANBK,9076562500,10072609700000,25722651200000,AR2025,NaN,NaN,NaN,2.293394e+13
3,HDFCBANK,5105325797,402332200000,32224695700000,AR2025,NaN,NaN,NaN,1.651468e+13
4,ICICIBANK,3564037855,6187340900000,11151521900000,AR2025,NaN,NaN,NaN,1.176310e+13
5,AXISBANK,3098620347,3581757300000,11410175700000,AR2025,NaN,NaN,NaN,9.286845e+12
6,KOTAKBANK,1988519342,6129009600000,9336198400000,AR2025,NaN,NaN,NaN,1.079711e+13
7,INDUSINDBK,779445161,2848660500000,3045799500000,AR2025,NaN,NaN,NaN,4.371560e+12
8,BAJFINANCE,6208203435,1085765100000,1683317300000,AR2025,NaN,NaN,NaN,1.927424e+12
9,PNB,11521086957,5895063500000,10608938500000,AR2025,NaN,NaN,NaN,1.119953e+13


In [31]:
mc_df = compute_marketcap_from_fundamentals(
    tickers=tickers,
    price_folder=price_folder,
    fundamentals_dict=fund,
    fy_end=fy_end
)

mc_df

,Ticker,AdjClose_FYend,FYEnd_Price_Date,Shares_Outstanding,Market_Cap_Rupees,Market_Cap_Crore
0,SBIBANK,756.313538,2025-03-28,8924620034,6.749811e+12,674981.094963
1,BANKBARODA,220.916595,2025-03-28,5171362179,1.142440e+12,114243.972647
2,CANBK,85.921478,2025-03-28,9076562500,7.798717e+11,77987.166762
3,HDFCBANK,901.908936,2025-03-28,5105325797,4.604539e+12,460453.895519
4,ICICIBANK,1338.025635,2025-03-28,3564037855,4.768774e+12,476877.401327
5,AXISBANK,1101.058960,2025-03-28,3098620347,3.411764e+12,341176.369658
6,KOTAKBANK,2168.699463,2025-03-28,1988519342,4.312501e+12,431250.082894
7,INDUSINDBK,649.849976,2025-03-28,779445161,5.065224e+11,50652.241885
8,BAJFINANCE,889.073914,2025-03-28,6208203435,5.519552e+12,551955.172422
9,PNB,93.422897,2025-03-28,11521086957,1.076333e+12,107633.332402


In [32]:
out_path = Path(cfg["output_folder"]) / "marketcap_FY2025.csv"
mc_df.to_csv(out_path, index=False)

print("✓ Market Cap calculation complete.")
print("Saved to:", out_path)

✓ Market Cap calculation complete.
Saved to: ../outputs/marketcap_FY2025.csv
